<a href="https://colab.research.google.com/github/gsnsg/DL/blob/master/PyTorch/PyTorch_MNIST_Classifaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import torch
import torchvision
import torch.nn as nn
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

In [18]:
train_dataset = MNIST('.', train=True, transform=torchvision.transforms.ToTensor() ,download=True)
test_dataset = MNIST('.', train=False, transform=torchvision.transforms.ToTensor(), download=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [30]:
hidden_dim = 128
model = nn.Sequential(
    nn.Linear(28 * 28, hidden_dim),
    nn.ReLU(inplace = True),
    nn.Linear(hidden_dim, hidden_dim),
    nn.ReLU(inplace=True),
    nn.Linear(hidden_dim, 10)
)

model = model.to(device)

In [32]:
train_dataloader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size = 64, shuffle = True)

In [33]:
import tqdm.auto as tqdm

criterion = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr=0.001)

n_epochs = 10
n_steps = len(train_dataset) // 64 

model.train()

for epoch in range(n_epochs):

  epoch_loss = 0

  for X, target in train_dataloader:
    cur_batch_size = X.shape[0]

    X = X.view(cur_batch_size, -1).to(device)
    target = target.to(device)

    pred = model(X)
    loss = criterion(pred, target)
    
    epoch_loss += loss.item()
    opt.zero_grad()
    loss.backward()
    opt.step()
  
  print(f'Epoch: {epoch + 1}, Loss: {epoch_loss}')






Epoch: 1, Loss: 298.0083209872246
Epoch: 2, Loss: 122.64717141818255
Epoch: 3, Loss: 81.12427589995787
Epoch: 4, Loss: 60.377453226130456
Epoch: 5, Loss: 47.07831170014106
Epoch: 6, Loss: 41.07251304562669
Epoch: 7, Loss: 32.123200696602
Epoch: 8, Loss: 26.627845267459634
Epoch: 9, Loss: 21.131691711081658
Epoch: 10, Loss: 18.810820294704172


In [41]:
num_correct =  0

model.eval()

with torch.no_grad():
  for X, target in test_dataloader:
    X = X.view(X.shape[0], -1).to(device)
    target = target.to(device)

    pred = model(X)
    vals, ind = torch.max(pred, axis = 1)

    num_correct += torch.sum(ind == target)

print("Accuracy: ", num_correct.item() / len(test_dataset))

Accuracy:  0.9754
